## Model Data using Cassandra


### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [59]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

In [89]:
# Reading the data
df = pd.read_csv('event_data.csv')
df.head()

,artist_name,fname,gender,item_in_session_number,lname,length,level,location,session_number,song_title,user_id
0,Pavement,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",345,Mercy:The Laundromat,10
1,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",438,Horn Concerto No. 4 in E flat K495: II. Romanc...,53
2,Gary Allan,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",438,Nothing On But The Radio,53
3,Charttraxx Karaoke,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",438,Fireflies,53
4,The Libertines,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",389,The Good Old Days,29


In [71]:
# Connecting with the server
session = Cluster(
    cloud={"secure_connect_bundle": "secure-connect-soundcloud-db.zip"},
    auth_provider=PlainTextAuthProvider("token", "AstraCS:qHJQpwYqKwQIljswscAWmWFo:3add6c200ac20cd82b176b773affdff494e79c08dbc28b58bf0570d138625164"),
).connect()
print(session)

In [90]:
# Checking the server connection and the keyspace 
val = session.execute(
    """
    SELECT * FROM system_schema.keyspaces;
    """
)
for i in val:
    print(i)

Row(keyspace_name='system_auth', durable_writes=True, graph_engine=None, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy'), ('us-east-2', '3')]))
Row(keyspace_name='system_schema', durable_writes=True, graph_engine=None, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.LocalStrategy')]))
Row(keyspace_name='data_endpoint_auth', durable_writes=True, graph_engine=None, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy'), ('us-east-2', '3')]))
Row(keyspace_name='datastax_sla', durable_writes=True, graph_engine=None, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy'), ('us-east-2', '3')]))
Row(keyspace_name='soundcloud', durable_writes=True, graph_engine=None, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy'), ('us-east-2', '3')]))
Row(keyspace_name='system'

In [74]:
# Creating Table 'events' in 'soundcloud' keyspace 

keyspace = 'soundcloud'

try:
  session.execute(
    f"""
    CREATE TABLE IF NOT EXISTS {keyspace}.events (
        {df.columns[0]} UUID,
        {df.columns[1]} TEXT,
        {df.columns[2]} TEXT,
        {df.columns[3]} TEXT,
        {df.columns[4]} INT,
        {df.columns[5]} TEXT,
        {df.columns[6]} FLOAT,
        {df.columns[7]} TEXT,
        {df.columns[8]} TEXT,
        {df.columns[9]} INT,
        {df.columns[10]} TEXT,
        {df.columns[11]} INT,
        PRIMARY KEY ((user_id, session_number), item_in_session_number)
    );
    """
)
  
except Exception as e:
  print("Error: ", e)


In [77]:
# loading the data into the table

query = f'INSERT INTO {keyspace}.events (artist_name, fname, gender, item_in_session_number, lname, length, level, location, session_number, song_title, user_id)'
query += f' VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'

for index, row in df.iterrows():
    try:
        session.execute(query, (row['artist_name'], row['fname'], row['gender'], row['item_in_session_number'], row['lname'], row['length'], row['level'], row['location'], row['session_number'], row['song_title'], row['user_id']))

    except Exception as e:
        print(f"Error inserting data into events table: {e}")
        break

print('Data entered successfully')

Data entered successfully


## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4







In [78]:
try:
   val = session.execute(
        f'''
            SELECT artist_name, song_title, length FROM {keyspace}.events
            where session_number = 338 AND item_in_session_number = 4 
            ALLOW FILTERING;
        '''
    )

except Exception as e:
   print(f"Error: {e}")

for i in val:
   print(i)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    



In [84]:
try:
   session.execute(
      f'''
        CREATE INDEX IF NOT EXISTS 
        ON {keyspace}.events(item_in_session_number);

      '''
   )
   
except Exception as e:
   print(f"An error occurred: {e}")

try:
   val = session.execute(
        f'''
            SELECT artist_name, song_title, fname, lname FROM {keyspace}.events
            WHERE user_id = 10 AND session_number = 182
            ORDER BY item_in_session_number
            ALLOW FILTERING;
        '''
    )

except Exception as e:
   print(f"Error: {e}")

for i in val:
   print(i)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", fname='Sylvie', lname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', fname='Sylvie', lname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', fname='Sylvie', lname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', fname='Sylvie', lname='Cruz')


### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'

In [86]:
try:
   val = session.execute(
        f'''
            SELECT fname, lname FROM {keyspace}.events
            WHERE song_title = 'All Hands Against His Own'
            ALLOW FILTERING;
        '''
    )

except Exception as e:
   print(f"Error: {e}")

for i in val:
   print(i)

Row(fname='Tegan', lname='Levine')
Row(fname='Jacqueline', lname='Lynch')
Row(fname='Sara', lname='Johnson')


### Dropping the table

In [91]:
try:
   session.execute(
        f'''
            DROP TABLE {keyspace}.events;
        '''
    )

except Exception as e:
   print(f"Error: {e}")

### Shutting down the connections

In [93]:
session.shutdown()